##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Import Packages

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Data Preprocessing

In [ ]:
# Hyperparameter related to loading our dataset
IMG_SIZE = (160, 160)

# We are going to download open-sourced dataset to demonstrate transfer learning
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True, cache_subdir='/content/')
PATH = f'{path_to_zip}/cats_and_dogs_filtered'

# Generate tf.data.Datset for training and validation dataset
# This allows us to efficiently load batch of data to our NN model for training and validation
# FYI: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir, shuffle=True, image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir, shuffle=True, image_size=IMG_SIZE)

## Dataset Splitting

Conventionally, researchers and engineers split the entire dataset into three groups: 1) Train dataset, 2) Validation dataset, 3) Test dataset. Typically, 1) is directly utilized to train our NN while 2) is used to log how the model learns during the training. On the other hand, 3) is literally used to test how well the model generalizes across new dataset, which is an important performance metric of NN models.

**Now, the question is: why are we introducing 2) during the training if it doesn't even used for the actual training process?**

In [ ]:
# The original dataset doesn't contain a test set.
# So, we are going to create one. To do so, we determine how many batches of data are available in the validation set using tf.data.experimental.cardinality.
# We will use 20% of the validation data as a test set.

val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

print(f'# of validation batches: {tf.data.experimental.cardinality(validation_dataset)}')
print(f'# of test batches: {tf.data.experimental.cardinality(test_dataset)}')

## Data Inspection & Visualization

In [ ]:
# Let's visualize several images in our dataset
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

## Rescale Pixel Values

In a moment, we will download `tf.keras.applications.MobileNetV2` for use as your base model. This model expects pixel values in `[-1, 1]`, but at this point, the pixel values in your images are in `[0, 255]`. Hence, we need to adjust this for training our own model.

In [ ]:
# To rescale them, we use the preprocessing method included with the model.
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

## Base Model (i.e., Backbone) Preparation

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,) # (160, 160, 3) -> RGB image
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=, weights='imagenet')

## Method 1. Feature Extraction

Feature Extraction is a method where you use a base model (i.e., **backbone**) as a generic feature extractor. This method requires the least amount of effort to generate your own NN as you only train a **head** for your own application.

In [ ]:
# This will "freeze" weights and biases in our base model.
# In other words, there will be no backpropagation of the loss graidents to update weights and biases in the NN.
base_model.trainable =

# Now, let's design our own classification head for our own application
# When generating your own NN model, make sure that intermediate dimenions between layers are correct!
# Let's first check output dimension of the base_model
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

# Since we are going to design a simple binary classifiation model, we will use average pooling layer to obtain a global feature of an inpute image.
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

# Then, make sure that we get the expected shape after going to our classification head.
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

# Finally, let's build our model.
# We will also check whether our model works as expected. We will use a dummy image to test the model.
inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Check how many parameters are trainable
model.summary()
len(model.trainable_variables)
tf.keras.utils.plot_model(model, show_shapes=True)

## Model Training

In [ ]:
base_learning_rate = 0.0001 # Another important hyperparameter to train our model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])


# Let's see how this process will improve our NN after 10 epochs of training.
epochs = 10
loss0, accuracy0 = model.evaluate(validation_dataset)

print(f"initial loss: {loss0:.2f}")
print(f"initial accuracy: {accuracy0:.2f}")

history = model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)

# Visualization of loss and accuracy during the training process
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Now, let's check how the model works on the test dataset
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")